In [2]:
from datasets import load_dataset, Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, TextClassificationPipeline
import torch
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
from evaluate import evaluator
import pandas as pd
from math import ceil, floor
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

2024-04-26 23:01:42.051653: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-26 23:01:42.051738: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-26 23:01:42.054037: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-26 23:01:42.357375: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-26 23:01:43.891933: W tensorflow/compiler/

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/leontioso/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
stop_words = set(stopwords.words('english'))

In [5]:
echr = load_dataset("ecthr_cases",  "violation-prediction")

In [6]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [7]:
def encode(examples):
    return tokenizer( examples["text"],
                     truncation=True, 
                     padding=True)

In [8]:
train_dataset, val_dataset, test_dataset = echr['train'], echr['validation'], echr['test']
train_dataset, val_dataset, test_dataset = [dataset.map( lambda examples: {"text": "\n".join(examples["facts"])}) for dataset in [train_dataset, val_dataset, test_dataset]]
train_dataset, val_dataset, test_dataset = [dataset.map(encode, batched=True) for dataset in [train_dataset, val_dataset, test_dataset]]

In [9]:
text = train_dataset['text'][1]
encoded_text = tokenizer.encode(text)
decoded_text = tokenizer.decode(encoded_text)

Token indices sequence length is longer than the specified maximum sequence length for this model (3506 > 512). Running this sequence through the model will result in indexing errors


In [10]:
decoded_text

'[CLS] 9. the applicant is the monarch of liechtenstein, born in 1945 and living in vaduz ( liechtenstein ). 10. the applicant ’ s late father, the former monarch of liechtenstein, had been the owner of the painting szene an einem romischen kalkofen ( alias der große kalkofen ) of pieter van laer, which had formed part of his family ’ s art collection since at least 1767. until the end of the second world war the painting had been in one of the family ’ s castles on the territory of the now czech republic. 11. in 1946 the former czechoslovakia confiscated the property of the applicant ’ s father which was situated in its territory, including the painting in question, under decree no. 12 on the “ confiscation and accelerated allocation of agricultural property of german and hungarian persons and of those having committed treason and acted as enemies of the czech and slovak people ” ( dekretu prezidenta republiky c. 12 / 1945 sb. o konfiskaci a urychlenem rozdeleni majetku nemcu, mad ’ a

In [11]:
train_df = pd.DataFrame(train_dataset)

In [13]:
train_df['text'].str.lower().str.split().apply(lambda x: [item for item in x if item not in stop_words]).apply(lambda x: " ".join(x)).replace('\d+', '', regex=True)

0       . beginning events relevant application, k. da...
1       . applicant monarch liechtenstein, born  livin...
2       . june  plots agricultural land owned applican...
3       .  mr dušan slobodník, research worker field l...
4       . applicant italian citizen, born  living oris...
                              ...                        
8995    . applicant born  lives village karakert, arme...
8996    . applicant born  currently detention regensdo...
8997    . applicants russian nationals who, material t...
8998    . cases applicants brought civil proceedings r...
8999    . applicant born  lives pécs. practising lawye...
Name: text, Length: 9000, dtype: object